In [13]:
%pip install numpy pandas nltk textblob statistics
import nltk
nltk.download('stopwords')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/atharvchandratre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [15]:
data = pd.read_csv('../dataset/dataset.csv', sep='|')
data.Reviews=data.Reviews.astype(str)

#Transform text to lowercase
data['Reviews'] = data['Reviews'].apply(lambda x: x.lower()) 
#Removing all punctuations and special characters
data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('[,.]', '', x))

In [16]:
stopwords_list = stopwords.words('english')
data['Reviews'] = data['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [17]:
print(data)

                   Company                                            Reviews
0                  Netflix  high performance culture challenge netflix ask...
1                     Uber  pay gone years first started pay great! gas pr...
2                     Yext  excellent ceo -lucky worked given opportunity ...
3                     CACI  nice place work great company work benefits gr...
4            Covenant Eyes  best company i’ve ever worked covenant eyes ca...
..                     ...                                                ...
117  Reynolds and Reynolds  5 stars best part working company? management ...
118                  Sabre  easy going workplace “you always deliver work ...
119  Legrand North America  best place work managers jobs correctly always...
120    Samsung Electronics  interesting challenging work one favorite comp...
121         Teladoc Health  well-organized inclusive excellent management ...

[122 rows x 2 columns]


In [18]:
from textblob import TextBlob

In [19]:
score_list=[]

for text in data['Reviews']:
  
  sentence = TextBlob(text)
  
  score = sentence.sentiment.polarity
  score_list.append(score)

In [20]:
import statistics
median = statistics.median(score_list)
print(median)

0.24523521411378624


In [21]:
sentiment_label = []
for score in score_list:
  if score > median:
    sentiment_label.append('positive')
  elif score < median:
    sentiment_label.append('negative')
  else:
    sentiment_label.append('neutral')
print(sentiment_label)

['positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'po

In [22]:
dataset = data
dataset['SCORE']=score_list
dataset['SENTIMENT_LABEL']=sentiment_label
print(dataset)
dataset.to_csv("../dataset/TextBlobResultswithMedian.csv", sep='|')

                   Company                                            Reviews  \
0                  Netflix  high performance culture challenge netflix ask...   
1                     Uber  pay gone years first started pay great! gas pr...   
2                     Yext  excellent ceo -lucky worked given opportunity ...   
3                     CACI  nice place work great company work benefits gr...   
4            Covenant Eyes  best company i’ve ever worked covenant eyes ca...   
..                     ...                                                ...   
117  Reynolds and Reynolds  5 stars best part working company? management ...   
118                  Sabre  easy going workplace “you always deliver work ...   
119  Legrand North America  best place work managers jobs correctly always...   
120    Samsung Electronics  interesting challenging work one favorite comp...   
121         Teladoc Health  well-organized inclusive excellent management ...   

        SCORE SENTIMENT_LAB

In [23]:
human_data = pd.read_csv('../dataset/HumanAnnotatedDataset.csv', sep=',')

Pos_Fal_neg = 0.0
Pos_Tru_pos = 0.0
Neg_Tru_neg = 0.0
Neg_Fal_pos = 0.0


for ind in range(len(sentiment_label)):
  
    if sentiment_label[ind] == 'positive' and human_data['HUMAN_LABEL'][ind] == 'negative':
        Neg_Fal_pos += 1
    elif sentiment_label[ind] == 'negative' and human_data['HUMAN_LABEL'][ind] == 'positive':
        Pos_Fal_neg += 1
    elif sentiment_label[ind] == 'positive' and human_data['HUMAN_LABEL'][ind] == 'positive':
        Pos_Tru_pos += 1
    elif sentiment_label[ind] == 'negative' and human_data['HUMAN_LABEL'][ind] == 'negative':
        Neg_Tru_neg += 1


Pos_Prec = (Pos_Tru_pos)/(Pos_Tru_pos + Neg_Fal_pos)
Pos_Recal = (Pos_Tru_pos)/(Pos_Tru_pos + Pos_Fal_neg)
Pos_FScore = (2*Pos_Prec*Pos_Recal)/(Pos_Prec + Pos_Recal)

Neg_Prec = (Neg_Tru_neg)/(Neg_Tru_neg + Pos_Fal_neg)
Neg_Recal = (Neg_Tru_neg)/(Neg_Tru_neg + Neg_Fal_pos)
Neg_FScore = (2*Neg_Prec*Neg_Recal)/(Neg_Prec + Neg_Recal)
        
    
print("Positive Precision: ", Pos_Prec)
print("Positive Recall: ", Pos_Recal)
print("Positive F-Score: ", Pos_FScore)

print("\n")

print("Negative Precision: ", Neg_Prec)
print("Negative Recall: ", Neg_Recal)
print("Negative F-Score: ", Neg_FScore)

Positive Precision:  0.6721311475409836
Positive Recall:  0.8913043478260869
Positive F-Score:  0.7663551401869158


Negative Precision:  0.9180327868852459
Negative Recall:  0.7368421052631579
Negative F-Score:  0.8175182481751824
